In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [48]:
#read data into dataframe
data = pd.read_csv('train.csv')


In [49]:
data.head()

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
0,TACOSSSSSSS,BROOKLYN,3398,FULTON ST,11208.0,7182773780,Mexican,6/8/2016,Violations were cited in the following area(s).,89.0,NaN,NaN,8/28/2017,Initial Inspection,Pre-permit (Non-operational),Critical,86566813
1,SPOLINI'S,QUEENS,11625,METROPOLITAN AVENUE,11418.0,7188055852,Italian,2/18/2016,Violations were cited in the following area(s).,23.0,NaN,NaN,8/28/2017,Initial Inspection,Cycle Inspection,Critical,84913823
2,EL DORADO CAFE,QUEENS,10202,ROOSEVELT AVENUE,11368.0,7184265992,"Latin (Cuban, Dominican, Puerto Rican, South &...",3/4/2015,Violations were cited in the following area(s).,7.0,A,3/4/2015,8/28/2017,Initial Inspection,Cycle Inspection,Critical,70749585
3,SANREMO PIZZA,QUEENS,5601,MYRTLE AVENUE,11385.0,7184971903,Pizza,12/4/2014,Violations were cited in the following area(s).,12.0,A,12/4/2014,8/28/2017,Initial Inspection,Cycle Inspection,Critical,59396548
4,95 SOUTH,BROOKLYN,778,FRANKLIN AVENUE,11238.0,3477506180,American,6/3/2017,Violations were cited in the following area(s).,11.0,NaN,NaN,8/28/2017,Initial Inspection,Inter-Agency Task Force,Not Critical,79915156


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215955 entries, 0 to 215954
Data columns (total 17 columns):
DBA                    215763 non-null object
BORO                   215955 non-null object
BUILDING               215890 non-null object
STREET                 215949 non-null object
ZIPCODE                215949 non-null float64
PHONE                  215951 non-null object
CUISINE DESCRIPTION    215955 non-null object
INSPECTION DATE        215955 non-null object
ACTION                 215342 non-null object
SCORE                  203548 non-null float64
GRADE                  105650 non-null object
GRADE DATE             104257 non-null object
RECORD DATE            215955 non-null object
INSPECTION NO.         215342 non-null object
INSPECTION_TYPE        215955 non-null object
CRITICAL FLAG          215955 non-null object
CAMIS                  215955 non-null int64
dtypes: float64(2), int64(1), object(14)
memory usage: 28.0+ MB


In [51]:
data.columns

Index(['DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION NO.', 'INSPECTION_TYPE',
       'CRITICAL FLAG', 'CAMIS'],
      dtype='object')

Missing Values

In [52]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [53]:
missing_values_table(data)

Your selected dataframe has 17 columns.
There are 10 columns that have missing values.


,Missing Values,% of Total Values
GRADE DATE,111698,51.7
GRADE,110305,51.1
SCORE,12407,5.7
ACTION,613,0.3
INSPECTION NO.,613,0.3
DBA,192,0.1
BUILDING,65,0.0
STREET,6,0.0
ZIPCODE,6,0.0
PHONE,4,0.0


In [54]:
data[data['SCORE'].isnull()].head()

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
12,FRANKLIN HOTEL,MANHATTAN,164,EAST 87 STREET,10128.0,2123691000,Continental,4/29/2015,Violations were cited in the following area(s).,NaN,NaN,NaN,8/28/2017,Initial Inspection,Administrative Miscellaneous,Not Critical,54875330
21,RICHMOND COUNTY YACHT CLUB,STATEN ISLAND,142,MANSION AVENUE,10308.0,7183564120,American,10/30/2015,Violations were cited in the following area(s).,NaN,NaN,NaN,8/28/2017,Initial Inspection,Administrative Miscellaneous,Not Critical,46213624
28,CRISPIN'S HELL'S KITCHEN,MANHATTAN,764,10TH AVE,10019.0,2125860888,American,11/9/2016,Violations were cited in the following area(s).,NaN,NaN,NaN,8/28/2017,Initial Inspection,Administrative Miscellaneous,Not Critical,39546299
30,BELLA ROZA,QUEENS,6352,BOOTH STREET,11374.0,7182750777,Jewish/Kosher,12/22/2015,No violations were recorded at the time of thi...,NaN,NaN,NaN,8/28/2017,Re-inspection,Administrative Miscellaneous,Not Applicable,73056311
54,SILVER KRUST WEST INDIAN RESTAURANT,BROOKLYN,747,NOSTRAND AVENUE,11216.0,7186042076,Caribbean,9/24/2016,Violations were cited in the following area(s).,NaN,NaN,NaN,8/28/2017,Initial Inspection,Smoke-Free Air Act,Not Critical,78321923


In [152]:
data.GRADE.unique()
data.groupby('GRADE')['CAMIS'].nunique()


GRADE
A                 83437
B                 15187
C                  3786
Not Yet Graded     1398
P                   696
Z                  1146
Name: CAMIS, dtype: int64

In [57]:
data[data['GRADE'] == 'Not Yet Graded'].count()

DBA                    1398
BORO                   1398
BUILDING               1398
STREET                 1398
ZIPCODE                1398
PHONE                  1398
CUISINE DESCRIPTION    1398
INSPECTION DATE        1398
ACTION                 1398
SCORE                  1397
GRADE                  1398
GRADE DATE                5
RECORD DATE            1398
INSPECTION NO.         1398
INSPECTION_TYPE        1398
CRITICAL FLAG          1398
CAMIS                  1398
dtype: int64

In [156]:
data.groupby('RECORD DATE')['CAMIS'].nunique()

RECORD DATE
8/28/2017    215955
Name: CAMIS, dtype: int64

In [187]:
data.groupby('INSPECTION NO.')['CAMIS'].nunique()


INSPECTION NO.
 Compliance Inspection             1557
 Initial Inspection              148906
 Limited Inspection                   1
 Re-inspection                    62719
 Reopening Inspection              2053
 Second Compliance Inspection       106
Name: CAMIS, dtype: int64

In [188]:
data.groupby('INSPECTION_TYPE')['CAMIS'].nunique()

INSPECTION_TYPE
Administrative Miscellaneous       5818
Calorie Posting                     414
Cycle Inspection                    613
Cycle Inspection                 180612
Inter-Agency Task Force             619
Pre-permit (Non-operational)       2285
Pre-permit (Operational)          20030
Smoke-Free Air Act                 3129
Trans Fat                          2435
Name: CAMIS, dtype: int64

In [190]:
data.groupby('ZIPCODE')['CAMIS'].nunique().count()

231

In [191]:
data.groupby('STREET')['CAMIS'].nunique().count()

3272

In [193]:
data.groupby('BORO')['CAMIS'].nunique()

BORO
BRONX            18802
BROOKLYN         53768
MANHATTAN        86228
Missing              6
QUEENS           49940
STATEN ISLAND     7211
Name: CAMIS, dtype: int64

In [ ]:
data.groupby('BORO')['CAMIS'].nunique()

In [58]:
columns_to_drop = ['GRADE','GRADE DATE','RECORD DATE','PHONE','STREET','BUILDING']

data.drop(labels = columns_to_drop,inplace=True,axis=1)

In [59]:
missing_values_table(data)

Your selected dataframe has 11 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
SCORE,12407,5.7
ACTION,613,0.3
INSPECTION NO.,613,0.3
DBA,192,0.1
ZIPCODE,6,0.0


Exploratory Data Analysis

In [143]:
data['INSPECTION NO.'].value_counts()
data['ACTION'].value_counts()

Violations were cited in the following area(s).                                                                                        205778
Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.      4917
No violations were recorded at the time of this inspection.                                                                              2787
Establishment re-opened by DOHMH                                                                                                         1411
Establishment re-closed by DOHMH                                                                                                          449
Name: ACTION, dtype: int64

In [62]:
data['SCORE'].median()

15.0

In [63]:
data['SCORE'].replace(np.nan,data['SCORE'].median(),inplace = True)
missing_values_table(data)


Your selected dataframe has 11 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
ACTION,613,0.3
INSPECTION NO.,613,0.3
DBA,192,0.1
ZIPCODE,6,0.0


In [234]:
data[data['DBA'].isnull()].

,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
3015,NaN,MANHATTAN,10029.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,73013663
4056,NaN,BRONX,10452.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,41164983
4614,NaN,STATEN ISLAND,10314.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,80553405
4722,NaN,MANHATTAN,10021.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,19043173
6202,NaN,MANHATTAN,10034.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,29939040


In [10]:
data[data['CRITICAL FLAG'] == 'Not Applicable'].head()

,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
30,BELLA ROZA,QUEENS,11374.0,Jewish/Kosher,12/22/2015,No violations were recorded at the time of thi...,18.902485,Re-inspection,Administrative Miscellaneous,Not Applicable,73056311
72,LAS JAROCHITAS RESTAURANT,QUEENS,11372.0,Spanish,10/25/2015,No violations were recorded at the time of thi...,18.902485,Initial Inspection,Trans Fat,Not Applicable,70117835
101,STARBUCKS,MANHATTAN,10017.0,Café/Coffee/Tea,9/14/2015,Violations were cited in the following area(s).,0.000000,Initial Inspection,Cycle Inspection,Not Applicable,87356299
182,PAK LAHORI CHILLI,BROOKLYN,11230.0,Indian,6/26/2015,No violations were recorded at the time of thi...,18.902485,Initial Inspection,Administrative Miscellaneous,Not Applicable,48806695
240,LONG SPRING GARDEN,QUEENS,11377.0,Chinese,8/20/2015,Violations were cited in the following area(s).,11.000000,Re-inspection,Pre-permit (Operational),Not Applicable,99192167


In [11]:
missing_values_table(data)

Your selected dataframe has 11 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
ACTION,613,0.3
INSPECTION NO.,613,0.3
DBA,192,0.1
ZIPCODE,6,0.0


In [255]:
data[data['ZIPCODE'].isnull()]

,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS


In [64]:
data.drop(axis = 0,inplace = True,index=data[data['ZIPCODE'].isnull()].index)


In [72]:
data[data['INSPECTION DATE']== '1/1/1900'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 444 to 215691
Data columns (total 11 columns):
DBA                    417 non-null object
BORO                   417 non-null object
ZIPCODE                417 non-null float64
CUISINE DESCRIPTION    417 non-null object
INSPECTION DATE        417 non-null object
ACTION                 417 non-null object
SCORE                  417 non-null float64
INSPECTION NO.         417 non-null object
INSPECTION_TYPE        417 non-null object
CRITICAL FLAG          417 non-null object
CAMIS                  417 non-null int64
dtypes: float64(2), int64(1), object(8)
memory usage: 39.1+ KB


In [14]:
data[data['CRITICAL FLAG']== 'Not Applicable'].info()
data[data['CRITICAL FLAG']== 'Not Applicable'].head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3659 entries, 30 to 215940
Data columns (total 11 columns):
DBA                    3469 non-null object
BORO                   3659 non-null object
ZIPCODE                3659 non-null float64
CUISINE DESCRIPTION    3659 non-null object
INSPECTION DATE        3659 non-null object
ACTION                 3052 non-null object
SCORE                  3659 non-null float64
INSPECTION NO.         3052 non-null object
INSPECTION_TYPE        3659 non-null object
CRITICAL FLAG          3659 non-null object
CAMIS                  3659 non-null int64
dtypes: float64(2), int64(1), object(8)
memory usage: 343.0+ KB


,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
30,BELLA ROZA,QUEENS,11374.0,Jewish/Kosher,12/22/2015,No violations were recorded at the time of thi...,18.902485,Re-inspection,Administrative Miscellaneous,Not Applicable,73056311
72,LAS JAROCHITAS RESTAURANT,QUEENS,11372.0,Spanish,10/25/2015,No violations were recorded at the time of thi...,18.902485,Initial Inspection,Trans Fat,Not Applicable,70117835
101,STARBUCKS,MANHATTAN,10017.0,Café/Coffee/Tea,9/14/2015,Violations were cited in the following area(s).,0.000000,Initial Inspection,Cycle Inspection,Not Applicable,87356299
182,PAK LAHORI CHILLI,BROOKLYN,11230.0,Indian,6/26/2015,No violations were recorded at the time of thi...,18.902485,Initial Inspection,Administrative Miscellaneous,Not Applicable,48806695
240,LONG SPRING GARDEN,QUEENS,11377.0,Chinese,8/20/2015,Violations were cited in the following area(s).,11.000000,Re-inspection,Pre-permit (Operational),Not Applicable,99192167


In [265]:
data[data['DBA'].isnull()].head()

,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
3015,NaN,MANHATTAN,10029.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,73013663
4056,NaN,BRONX,10452.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,41164983
4614,NaN,STATEN ISLAND,10314.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,80553405
4722,NaN,MANHATTAN,10021.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,19043173
6202,NaN,MANHATTAN,10034.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,29939040


In [67]:
data.drop(axis = 0,inplace = True,index=data[data['DBA'].isnull()].index)

In [68]:
missing_values_table(data)

Your selected dataframe has 11 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
ACTION,417,0.2
INSPECTION NO.,417,0.2


In [283]:
data[data['ACTION'].isnull()].head()

,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
444,MIGHTY QUINN'S BARBEQUE,MANHATTAN,10036.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,83287556
759,FLORA,BROOKLYN,11215.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,90022764
3110,INTREPID SEA AIR & SPACE MUSEUM (WELCOME CENTER),MANHATTAN,10036.0,American,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,23019929
5581,SUBWAY 22574,MANHATTAN,10018.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,29132595
6020,THE DUMPLING SHOP,BRONX,10469.0,Other,1/1/1900,NaN,18.902485,NaN,Cycle Inspection,Not Applicable,57000466


In [73]:
data['ACTION'].replace(np.nan,data['ACTION'].mode()[0],inplace = True)
data['INSPECTION NO.'].replace(np.nan,data['INSPECTION NO.'].mode()[0],inplace = True)
#data['INSPECTION NO.'].mode()[0]

In [74]:
missing_values_table(data)

Your selected dataframe has 11 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215759 entries, 0 to 215954
Data columns (total 11 columns):
DBA                    215759 non-null object
BORO                   215759 non-null object
ZIPCODE                215759 non-null float64
CUISINE DESCRIPTION    215759 non-null object
INSPECTION DATE        215759 non-null object
ACTION                 215759 non-null object
SCORE                  215759 non-null float64
INSPECTION NO.         215759 non-null object
INSPECTION_TYPE        215759 non-null object
CRITICAL FLAG          215759 non-null object
CAMIS                  215759 non-null int64
dtypes: float64(2), int64(1), object(8)
memory usage: 29.8+ MB


In [28]:
data.head()

,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,SCORE,INSPECTION NO.,INSPECTION_TYPE,CRITICAL FLAG,CAMIS
0,TACOSSSSSSS,BROOKLYN,11208.0,Mexican,6/8/2016,Violations were cited in the following area(s).,89.0,Initial Inspection,Pre-permit (Non-operational),Critical,86566813
1,SPOLINI'S,QUEENS,11418.0,Italian,2/18/2016,Violations were cited in the following area(s).,23.0,Initial Inspection,Cycle Inspection,Critical,84913823
2,EL DORADO CAFE,QUEENS,11368.0,"Latin (Cuban, Dominican, Puerto Rican, South &...",3/4/2015,Violations were cited in the following area(s).,7.0,Initial Inspection,Cycle Inspection,Critical,70749585
3,SANREMO PIZZA,QUEENS,11385.0,Pizza,12/4/2014,Violations were cited in the following area(s).,12.0,Initial Inspection,Cycle Inspection,Critical,59396548
4,95 SOUTH,BROOKLYN,11238.0,American,6/3/2017,Violations were cited in the following area(s).,11.0,Initial Inspection,Inter-Agency Task Force,Not Critical,79915156


In [75]:
X = data.loc[:, data.columns != 'CRITICAL FLAG']
Y = data['CRITICAL FLAG']


In [38]:
X.head()
Y.head()

0        Critical
1        Critical
2        Critical
3        Critical
4    Not Critical
Name: CRITICAL FLAG, dtype: object

In [76]:
print(X.shape,Y.shape)

(215759, 10) (215759,)


In [79]:
#creating testing and training set

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.2,stratify = Y)

In [80]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(172607, 10) (43152, 10)
(172607,) (43152,)


Data Transformation
